In [ ]:
'''
import os

class dataloader(object):
    def __init__(self, input_dir, output_dir, training_size = 99):
    # Initialize an empty dictionary to store the data
        data_dict = {}
        data_loaded = 0
        # Loop through all files in the input directory
        for filename in os.listdir(input_dir):
            
            if filename.endswith(".jpg"):
                # Get the full path of the image file
                img_path = os.path.join(input_dir, filename)
                
                # Construct the corresponding txt file name
                txt_filename = filename.replace(".jpg", ".txt")
                txt_filename = txt_filename.replace("input", "output")
                txt_path = os.path.join(output_dir, txt_filename)
                
                if os.path.exists(txt_path):
                    with open(txt_path, 'r') as txt_file:
                        label = txt_file.read().strip()             
                        # Store the image path and label in the dictionary
                        data_dict[img_path] = label
                        data_loaded += 1
                        print(f"Pairing {filename} with {txt_filename}")
                        if data_loaded == training_size:
                            break

    
        self.data_dict = data_dict

'''

In [ ]:
'''import os
import langchain
from langchain.chains import TransformChain
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_core.runnables import chain
import base64
import json
from langchain.tools import tool


openai_api_key = "sk-proj-EdTJfI06sx021wOqwqMUT3BlbkFJQqpltq3qDR6x4yVxElu2"

class llm_model(object):
    def __init__(self, vision_prompt, few_shot_prompt, finetune = True, training_data = None):
        self.llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0.0, model="gpt-4o", max_tokens=1024)
        self.format_instruction = format_instruction = """
             Provide your output in the format of a python dict object code {"InferredCharacters": "XXXXX"}.
             Do not provide any other information. 
             Do not provide the backticks or any characters not related to the dict string such that I can convert to dict object directly by json.load(result)"""
        self.vision_prompt = vision_prompt 
        if finetune:
            self.vision_prompt = self.vision_prompt + few_shot_prompt
            for data_path in training_data.data_dict.keys():
                encoded_image_data = encode_image(data_path)
                fine_tune_data = f"""```Encoded image data: {encoded_image_data} -> True label: {training_data.data_dict[data_path]}``` \n"""
                self.vision_prompt = self.vision_prompt + fine_tune_data



def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
            
def load_image(inputs: dict) -> dict:
    """Load image from file and encode it as base64."""
    image_path = inputs["image_path"]
    image_base64 = encode_image(image_path)
    return {"image": image_base64}

load_image_chain = TransformChain(
    input_variables=["image_path"],
    output_variables=["image"],
    transform=load_image
)


finetune_input_dir = r"C:\Users\guang\OneDrive\Desktop\IMDA\fine-tune-data\input"
finetune_output_dir = r"C:\Users\guang\OneDrive\Desktop\IMDA\fine-tune-data\output"
training_data = dataloader(input_dir = finetune_input_dir, output_dir = finetune_output_dir)
llm_agent = llm_model(vision_prompt, few_shot_prompt, finetune = True, training_data = training_data)

@chain
def image_model(inputs: dict) -> str:
    """Invoke model with image and prompt."""
    model = llm_agent.llm
    # print("Final vision prompt\n", inputs["prompt"])
    msg = model.invoke(
             [SystemMessage(content=llm_agent.format_instruction),
              HumanMessage(
             content=[
             {"type": "text", "text": inputs["prompt"]},
             {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{inputs['image']}"}},
             ])]
             )
    return msg.content

@tool
def convert_output_to_dict(llm_output:str) -> dict:
    """Convert the output of the LLM to a simple dict with only the predicted output as the key"""
    try:
        response_dict = json.loads(llm_output)
    except json.JSONDecodeError:
        response_dict = {"error": "Invalid JSON response"}
    return response_dict
    


def get_image_information(image_path: str, llm_model) -> dict:
    vision_prompt = llm_model.vision_prompt
    vision_chain = load_image_chain | image_model | convert_output_to_dict
    return vision_chain.invoke({'image_path': f'{image_path}', 
                               'prompt': vision_prompt})
'''

In [ ]:
'''import os
import langchain
from langchain.chains import TransformChain
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_core.runnables import chain
import base64
import json
from langchain.tools import tool
from InferenceDataloader import dataloader


# Prompt Engineering portion

vision_prompt = """Given the Captcha image, where 
They resemble each other very much where the identify that the texture, nature of the font, spacing of the font, morphological characteristic of the letters and numerals arev very consistent.
Infer the image's characters (Only capital letters and numbers)

"""

few_shot_prompt = """Few shot prompting fine tunning: it is very important to take note of the difference between:
0 and O  

and

1 and I

In our dataset, their difference are consistent ('O' being more circular than '0' and having more rounding pixels at the top and bottom tip of the character and 'I' having a longer flat surface at the tip of the character than '1') and you must fine-tune yourself to adapt to the font format.
Pay attention to the top tip of the character when trying to distinguish these four characters.
A list of sample data describing the difference between '0', 'O', '1' and 'I' are provided to allow you to fine-tune your performance for the given task: 


"""


class llm_model(object):
    def __init__(self, vision_prompt, few_shot_prompt, finetune = True, training_data = None):
        self.llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0.0, model="gpt-4o", max_tokens=1024)
        self.format_instruction = format_instruction = """
             Provide your output in the format of a python dict object code {"InferredCharacters": "XXXXX"}.
             Do not provide any other information. 
             Do not provide the backticks or any characters not related to the dict string such that I can convert to dict object directly by json.load(result)"""
        self.vision_prompt = vision_prompt 
        if finetune:
            self.vision_prompt = self.vision_prompt + few_shot_prompt
            for data_path in training_data.data_dict.keys():
                encoded_image_data = encode_image(data_path)
                fine_tune_data = f"""```Encoded image data: {encoded_image_data} -> True label: {training_data.data_dict[data_path]}``` \n"""
                self.vision_prompt = self.vision_prompt + fine_tune_data

def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
            
def load_image(inputs: dict) -> dict:
    """Load image from file and encode it as base64."""
    image_path = inputs["image_path"]
    image_base64 = encode_image(image_path)
    return {"image": image_base64}

load_image_chain = TransformChain(
    input_variables=["image_path"],
    output_variables=["image"],
    transform=load_image
)

finetune_input_dir = r"C:\Users\guang\OneDrive\Desktop\IMDA\fine-tune-data\input"
finetune_output_dir = r"C:\Users\guang\OneDrive\Desktop\IMDA\fine-tune-data\output"
training_data = dataloader(input_dir = finetune_input_dir, output_dir = finetune_output_dir)
openai_api_key = "sk-proj-EdTJfI06sx021wOqwqMUT3BlbkFJQqpltq3qDR6x4yVxElu2"
llm_agent = llm_model(vision_prompt, few_shot_prompt, finetune = True, training_data = training_data)



@chain
def image_model(inputs: dict) -> str:
    """Invoke model with image and prompt."""
    model = llm_agent.llm
    # print("Final vision prompt\n", inputs["prompt"])
    msg = model.invoke(
             [SystemMessage(content=llm_agent.format_instruction),
              HumanMessage(
             content=[
             {"type": "text", "text": inputs["prompt"]},
             {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{inputs['image']}"}},
             ])]
             )
    return msg.content

@tool
def convert_output_to_dict(llm_output:str) -> dict:
    """Convert the output of the LLM to a simple dict with only the predicted output as the key"""
    try:
        response_dict = json.loads(llm_output)
    except json.JSONDecodeError:
        response_dict = {"error": "Invalid JSON response"}
    return response_dict


#OpenAI Inference Engine
class load_OpenAI_ai_model(object):
    def __init__(self):
        self.llm_agent = llm_agent

    def get_image_information(self, image_path: str, llm_model) -> dict:
        vision_prompt = self.llm_agent.vision_prompt
        vision_chain = load_image_chain | image_model | convert_output_to_dict
        return vision_chain.invoke({'image_path': f'{image_path}', 
                                   'prompt': vision_prompt})
'''

In [ ]:
# Sample usage of OpenAI trained torch model
import OpenAIInferenceEngine
from OpenAIInferenceEngine import load_OpenAI_ai_model, llm_model
from InferenceDataloader import dataloader

from PIL import Image

openai_api_key = "sk-proj-OP2IckZNWS1a9GsDQLYzT3BlbkFJkQ3fVVD3BpJw4cG8b6Yv"


# Use data used for zero-shot prompting to fine-tune LLM
finetune_input_dir = r"C:\Users\guang\OneDrive\Desktop\IMDA\fine-tune-data\input"
finetune_output_dir = r"C:\Users\guang\OneDrive\Desktop\IMDA\fine-tune-data\output"
training_data = dataloader(input_dir = finetune_input_dir, output_dir = finetune_output_dir)
OpenAIInferenceEngine.llm_agent = llm_model(finetune = False, training_data = training_data, openai_api_key=openai_api_key)


openai_ai_model = load_OpenAI_ai_model()
sample_catpcha_image_path = r'C:\Users\guang\OneDrive\Desktop\IMDA\sampleCaptchas\input\input19.jpg'

ir = openai_ai_model.get_image_information(sample_catpcha_image_path)
print(ir)

sample_image = Image.open(sample_catpcha_image_path)
display(sample_image)

In [ ]:
# Sample usage of torch trained torch model
from TorchInferenceEngine import load_torch_ai_model

from PIL import Image

torch_ai_model = load_torch_ai_model(model_path = 'imda_technical_test_pytorch_model.pth')
sample_catpcha_image_path = r'C:\Users\guang\OneDrive\Desktop\IMDA\sampleCaptchas\input\input19.jpg'

ir = torch_ai_model.get_image_information(sample_catpcha_image_path)
print(ir)

sample_image = Image.open(sample_catpcha_image_path)
display(sample_image)

In [1]:
"""Available Strategies:
1. OpenAI GPT4o AI Assistant - LLMAIAssistant
2. PyTorch self trained model - PytorchCNN"""

class Captcha(object):
    def __init__(self, strategy = ""):
        if strategy == "LLMAIAssistant":
            import OpenAIInferenceEngine
            from OpenAIInferenceEngine import load_OpenAI_ai_model, llm_model
            from InferenceDataloader import dataloader
            self.openai_ai_model = load_OpenAI_ai_model()
            self.caller = openai_ai_model.get_image_information
        
        elif strategy == "PytorchCNN":
            from TorchInferenceEngine import load_torch_ai_model
            self.torch_ai_model = load_torch_ai_model(model_path = 'imda_technical_test_pytorch_model.pth')
            self.caller = self.torch_ai_model.get_image_information
        else:
            print(f"{strategy} is not a valid method")

    def __call__(self, im_path, save_path):
        """
        Algo for inference
        args:
            im_path: .jpg image path to load and to infer
            save_path: output file path to save the one-line outcome
        """
        result = self.caller(im_path)
        return result


In [3]:
from PIL import Image

captcha_inferencer = Captcha(strategy = "PytorchCNN")
# declare inferencer and sample inferencing
sample_image = "C:/Users/guang/OneDrive/Desktop/IMDA/sampleCaptchas/input/input00.jpg"
result = captcha_inferencer(sample_image, None)
print(result)
displayable_image = Image.open(sample_image)
display(displayable_image)

{'InferredCharacters': 'EGGK4'}


In [4]:
input_dir = r"C:\Users\guang\OneDrive\Desktop\IMDA\sampleCaptchas\input"
output_dir = r"C:\Users\guang\OneDrive\Desktop\IMDA\sampleCaptchas\output"

inference_data = dataloader(input_dir, output_dir)

corrects = 0
attempts = 0
for data_path in inference_data.data_dict:
    true_label = inference_data.data_dict[data_path]
    inferred_output = captcha_inferencer(data_path, None)
    predict = inferred_output['InferredCharacters']
    print(f"Predict result: {predict} versus True Label: {true_label}")
    if predict == true_label:
        corrects += 1
    else:
        print("incorrect! Predicting", data_path)
    attempts += 1

print(f"Final remarks: corrects {corrects} out of {attempts} attempts. accuracy of {corrects/attempts:.2f}")


# Common errors, mistaken O as 0, I mistake as 1, mistake 0 as O, 

NameError: name 'dataloader' is not defined